# Part 1: Sequential Conversation - LangChain (Code-First)
This notebook demonstrates how to implement sequential conversation tracking using LangChain with memory.

**Goal:** Maintain context across at least 3 related conversation turns.

In [ ]:
# Install dependencies (if not already installed)
!pip install python-dotenv langchain-core langchain-community openai langchain tiktoken serpapi google-search-results sympy

## Setup OpenAI API Key

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
# Ensure you have your OpenAI API key in a .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## Initialize LangChain LLM with Memory

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

C:\Users\User\AppData\Local\Temp\ipykernel_39308\1740675483.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)
C:\Users\User\AppData\Local\Temp\ipykernel_39308\1740675483.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
C:\Users\User\AppData\Local\Temp\ipykernel_39308\1740675483.py:7: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain

## Test: Happy Path (Three Related Turns)

In [4]:
print(conversation.predict(input="Is there an outlet in Petaling Jaya?"))
print(conversation.predict(input="SS 2, what's the opening time?"))
print(conversation.predict(input="Do they offer delivery?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Is there an outlet in Petaling Jaya?
AI:

> Finished chain.
Yes, there are many outlets in Petaling Jaya. Some popular shopping malls in Petaling Jaya like 1 Utama, Sunway Pyramid, and Paradigm Mall have numerous outlets for various brands and products. Additionally, there are also standalone outlets for specific brands or stores scattered throughout the city. If you're looking for a specific type of outlet, feel free to ask for more details!


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the 

## Test: Interrupted Path (User changes topic midway)

In [8]:
conversation.memory.clear()
print(conversation.predict(input="Tell me about ZUS rewards program."))
print(conversation.predict(input="What about the outlet in Penang?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[]
Human: Tell me about ZUS rewards program.
AI:

> Finished chain.
ZUS rewards program is a loyalty program offered by ZUS, a popular retail store. Customers can earn points for every purchase they make at ZUS, and these points can be redeemed for discounts on future purchases. The program also offers exclusive deals, early access to sales, and special promotions for members. Overall, it's a great way for customers to save money and get rewarded for their loyalty to ZUS.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from 

# Part 2: Agentic Planning - LangChain (Code-First)

This notebook demonstrates how to implement a simple planner/controller loop using LangChain.

**Goal:** Parse user intent, detect missing information, and dynamically choose the appropriate next action:
- Ask a follow-up question
- Call a tool (e.g., calculator, RAG)
- Return a direct LLM response

In [4]:
# === Imports ===
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.llm import LLMChain
from langchain.tools import Tool
from langchain.utilities import SerpAPIWrapper
from langchain.tools import tool
from sympy import sympify
import re

os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

In [5]:
# === Setup Tools ===
@tool
def calculator(expression: str) -> str:
    """Evaluates a basic math expression."""
    match = re.search(r'(\d[\d\s\+\-\*/\.\(\)]*\d)', expression)

    if match:
        try:
            expr = match.group(1)
            result = sympify(expr)
            return str(result)
        except Exception as e:
            return f"Error: {str(e)}"
    else:
        print("Could not extract a valid math expression.")


search_tool = Tool(
    name="Search",
    func=SerpAPIWrapper().run,
    description="Useful for answering general knowledge or factual questions"
)

In [6]:
# === Setup LLM and Prompt ===
llm = ChatOpenAI(model="gpt-4", temperature=0)

intent_prompt = PromptTemplate.from_template("""
You are a smart assistant. Classify the user's intent as one of:
- "math"
- "search"
- "chat"

Then, identify if any information is missing to complete the task.
User: {input}
Respond in JSON with \"intent\" and \"missing_info\".
""")

intent_chain = LLMChain(prompt=intent_prompt, llm=llm, output_parser=StrOutputParser())


C:\Users\User\AppData\Local\Temp\ipykernel_39308\1555041687.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  intent_chain = LLMChain(prompt=intent_prompt, llm=llm, output_parser=StrOutputParser())


In [7]:
def beautify_search_output(raw_result):
    # If the result is a string representation of a list, convert it to a string
    if isinstance(raw_result, list):
        text = "\n".join(raw_result)
    else:
        # Try to safely eval if it's a string list, else just use as is
        try:
            import ast
            parsed = ast.literal_eval(raw_result)
            if isinstance(parsed, list):
                text = "\n".join(parsed)
            else:
                text = str(raw_result)
        except Exception:
            text = str(raw_result)
    # Use the LLM to beautify
    prompt = f"Summarize and present the following information in a user-friendly, readable way for a general audience:\n\n{text}"
    return llm.invoke(prompt).content

def agentic_planner(user_input: str):
    parsed = intent_chain.run({"input": user_input})
    print(f"[Intent Parser] Output: {parsed}")

    import json
    parsed_json = json.loads(parsed)
    intent = parsed_json.get("intent")
    missing_info = parsed_json.get("missing_info", "")

    if missing_info:
        print(f"Summary: I need more information to proceed: {missing_info}")
        return f"I need more information: {missing_info}"

    if intent == "math":
        answer = calculator.run(user_input)
        print("Summary: Calculation complete.")
        return f"Answer is {answer}"
    elif intent == "search":
        result = search_tool.run(user_input)
        beautified = beautify_search_output(result)
        print("Summary: Search action performed and information retrieved for the user.")
        return f"{beautified}"
    else:
        return f"I'm here to chat! Here's what I think: {llm.invoke(user_input)}"

In [53]:
# === Example Usage ===
print(agentic_planner("What's 25 * 4?"),'\n')
print(agentic_planner("Tell me about LangChain."),'\n')
print(agentic_planner("Book a flight"),'\n')
print(agentic_planner("What's the weather in Tokyo?"),'\n')

[Intent Parser] Output: {
  "intent": "math",
  "missing_info": null
}
Summary: Calculation complete.
Answer is 100 

[Intent Parser] Output: {
  "intent": "search",
  "missing_info": null
}
Summary: Search action performed and information retrieved for the user.
LangChain is a software framework designed to simplify the creation and integration of large language models (LLMs) into applications. It was initially released in October 2022 by developer Harrison Chase and is licensed under the MIT License. The software is written in Python and JavaScript and its repository can be found at github.com/langchain-ai/langchain. 

LangChain provides AI developers with tools to connect language models with external data sources. It is an open-source framework supported by an active community, making it a popular choice for developers and enterprises to build AI apps from prototype to production. 

The software is essentially a library of abstractions for Python and JavaScript, representing common

# Part 3: Tool Calling 

Objective: Integrate a Calculator Tool for simple arithmetic. 

Agents must detect arithmetic intent, invoke the calculator endpoint, parse responses, and 
handle errors without crashing. 
Deliverables: 

- Calculator API integration code or definition 
- Example transcripts showing both successful calculations and graceful failure 
handling

In [8]:
# === Setup Tools ===
@tool
def calculator(expression: str) -> str:
    """Evaluates a basic math expression."""
    match = re.search(r'(\d[\d\s\+\-\*/\.\(\)]*\d)', expression)

    if match:
        try:
            expr = match.group(1)
            result = sympify(expr)
            # Check for zoo (complex infinity)
            if str(result) == "zoo":
                return "undefined (division by zero)."
            return str(result)
        except Exception as e:
            return f"Error: {str(e)}"
    else:
        print("Could not extract a valid math expression.")


In [60]:
# === Testing Usage ===
print(agentic_planner("What's 2 + ?"),'\n')
print(agentic_planner("What's the answer of 2**16"),'\n')
print(agentic_planner("Solve the expression: 5/0"),'\n')

[Intent Parser] Output: {
  "intent": "math",
  "missing_info": "Second number in the equation"
}
Summary: I need more information to proceed: Second number in the equation
I need more information: Second number in the equation 

[Intent Parser] Output: {
  "intent": "math",
  "missing_info": null
}
Summary: Calculation complete.
Answer is 65536 

[Intent Parser] Output: {
  "intent": "math",
  "missing_info": null
}
Summary: Calculation complete.
Answer is undefined (division by zero). 



# Part 4: Combined API - `/products` (RAG with Pinecone) + `/outlets` (Text2SQL)
This notebook demonstrates how to:
- Ingest ZUS drinkware data into Pinecone
- Use LangChain for RetrievalQA over `/products`
- Use LangChain Text2SQL over Supabase or SQLite for `/outlets`
- Serve both as FastAPI endpoints

In [10]:
import sys
import os
from pathlib import Path
import pandas as pd

# Get the current notebook directory and add src to path
notebook_dir = Path.cwd()
src_path = notebook_dir.parent / 'src'
sys.path.append(str(src_path))

In [12]:
from scrape_zus import ZUSScraper

# # Use context manager (recommended)
scraper=ZUSScraper(headless=True, base_delay=2)
scraper.scrape_products(product_scrape=True, to_sql=True)

## Scrapping the outlets data into *.sql
- Handling for Consent/CAPTCHA -> Divide the outlets data (to be scraped) by 3 (processing seperately)

In [ ]:
outlets_df=pd.read_csv(os.path.join(scraper.save_path, "zus_outlets.csv"))
split_size = len(outlets_df) // 3
outlets_1 = outlets_df.iloc[:split_size]
outlets_2 = outlets_df.iloc[split_size:2*split_size]
outlets_3 = outlets_df.iloc[2*split_size:]

failed_outlets=[]

In [14]:
failed_outlets+=scraper.scrape_outlets(gmap_scrape=outlets_2)


Processing outlet id: <built-in function id> - Name: ZUS Coffee – Cubic Botanical, Pantai Dalam
Processing outlet link:  https://maps.app.goo.gl/2CFVnP1RUU76rsmZ8
CSV header already exists, skipping header write.

Processing outlet id: <built-in function id> - Name: ZUS Coffee – TTDI Grove Kajang
Processing outlet link:  https://maps.app.goo.gl/9RQRxmiGig3X6BXF9
CSV header already exists, skipping header write.

Processing outlet id: <built-in function id> - Name: ZUS Coffee – Jenjarom
Processing outlet link:  https://maps.app.goo.gl/jij7okFL6ZQ6oDj38
CSV header already exists, skipping header write.

Processing outlet id: <built-in function id> - Name: ZUS Coffee – Suasana PjH Presint 2
Processing outlet link:  https://maps.app.goo.gl/KuSAsowCiPCM3ZaY7
CSV header already exists, skipping header write.

Processing outlet id: <built-in function id> - Name: ZUS Coffee – Plaza Low yat
Processing outlet link:  https://maps.app.goo.gl/hP7wXLwkdTDEP9Jk8
CSV header already exists, skipping h

In [15]:
failed_outlets+=scraper.scrape_outlets(gmap_scrape=outlets_3)


Processing outlet id: <built-in function id> - Name: ZUS Coffee – Putrajaya Presint 15
Processing outlet link:  https://goo.gl/maps/LWyKNHec3hn7GTRKA
CSV header already exists, skipping header write.

Processing outlet id: <built-in function id> - Name: ZUS Coffee – Bandar Sungai Long
Processing outlet link:  https://goo.gl/maps/JUW33rY5syUx5wPJ9
CSV header already exists, skipping header write.

Processing outlet id: <built-in function id> - Name: ZUS Coffee – Quayside Mall
Processing outlet link:  https://goo.gl/maps/eHojDSUCoDMikY7S8
CSV header already exists, skipping header write.

Processing outlet id: <built-in function id> - Name: ZUS Coffee – MYDIN USJ Subang Jaya
Processing outlet link:  https://goo.gl/maps/Xx5HyDKByC969rev8
CSV header already exists, skipping header write.

Processing outlet id: <built-in function id> - Name: ZUS Coffee – Temu @ City Of Elmina
Processing outlet link:  https://goo.gl/maps/o3iKTrg6rGuo7Nh77
CSV header already exists, skipping header write.

P

In [20]:
# Check failed outlets
matched_1 = outlets_1[outlets_1["link"].isin(failed_outlets)]
matched_2 = outlets_2[outlets_2["link"].isin(failed_outlets)]
matched_3 = outlets_3[outlets_3["link"].isin(failed_outlets)]

# Concatenate them into one DataFrame
failed_df = pd.concat([matched_1, matched_2, matched_3], ignore_index=True)
failed_df

,name,address,link
0,ZUS Coffee – KL Eco City Mall,"G-12, Ground Floor, KL Eco City Mall, INC, 3, ...",https://maps.app.goo.gl/kFzmFBpFo1N9Qyi58
1,ZUS Coffee – Elmina Lakeside Mall,"GF-95 & 96, Elmina Lakeside Mall, No.5 & No.7,...",https://maps.app.goo.gl/R3BUrKfqbMsDcvyD7
2,ZUS Coffee – OUG Parklane,"A1-0-12 (Ground Floor), OUG Parklane, Jalan 1/...",https://maps.app.goo.gl/W24LoGds3JoiD3m37


### Manually edit the following link
- ZUS Coffee – KL Eco City Mal -> Don't have information in google map
- ZUS Coffee – Elmina Lakeside Mall -> Update data
- ZUS Coffee – OUG Parklane	-> Update data

In [ ]:
df = pd.read_csv("data/zus_outlets_final.csv", encoding='utf-8-sig')
df = df[df["name"] != "ZUS Coffee – KL Eco City Mall"]

target_name = "ZUS Coffee – Elmina Lakeside Mall"
df.loc[df["name"] == target_name, "link"] = "https://maps.app.goo.gl/P2KFUVb1eBvgV4Sg6"
df.loc[df["name"] == target_name, "reviews_count"] = 89
df.loc[df["name"] == target_name, "reviews_average"] = 4.7
df.loc[df["name"] == target_name, "phone_number"] = "012-816 1340"
df.loc[df["name"] == target_name, "services"] = "Dine-in, Takeaway, No-contact delivery"
df.loc[df["name"] == target_name, "place_type"] = "Cafe"
df.loc[df["name"] == target_name, "opens_at"] = "Monday, 10am–9:40pm, Tuesday, 10am–9:40pm, Wednesday, 10am–9:40pm, Thursday, 10am–9:40pm, Friday, 10am–9:40pm, Saturday, 10am–9:40pm, Sunday, 10am–9:40pm"

target_name = "ZUS Coffee – OUG Parklane"
df.loc[df["name"] == target_name, "reviews_count"] = 115
df.loc[df["name"] == target_name, "reviews_average"] = 4.4
df.loc[df["name"] == target_name, "phone_number"] = "N/A"
df.loc[df["name"] == target_name, "services"] = "Dine-in, Takeaway, No-contact delivery"
df.loc[df["name"] == target_name, "place_type"] = "Cafe"
df.loc[df["name"] == target_name, "opens_at"] = "Monday, 8 am–10:40 pm, Tuesday, 8 am–10:40 pm, Wednesday, 8 am–10:40 pm, Thursday, 8 am–10:40 pm, Friday, 8 am–10:40 pm, Saturday, 8 am–10:40 pm, Sunday, 8 am–10:40 pm"

duplicate_rows = df[df.duplicated()]
display(duplicate_rows)
df = df.drop_duplicates(keep='first').reset_index(drop=True)
df.to_csv("data/zus_outlets_final.csv", index=False, encoding="utf-8-sig")


In [19]:
scraper.scrape_outlets(to_sql=True)

Outlet scraping complete. SQL written to ../data/zus_outlets.sql


The `main.ipynb` notebook is already set up for LangChain experiments and has cells for both code and markdown explanations. To draft your Part 4 (RAG + Text2SQL API) work, you can add new cells for:
1. Product Vector Store Ingestion (Pinecone)
2. Product Retrieval Demo
3. Outlets SQLite DB Creation
4. Text2SQL Demo for Outlets
Below is a draft of the notebook cells you should add. You can copy-paste these into new cells in main.ipynb to prototype and test your logic.

In [ ]:
# Count Token 
df = pd.read_csv("/data/zus_products.csv")
df.insert(0, "id", range(1, len(df) + 1))
df.head()
# Step 2: Convert each row into a single string
docs = [
    f"Product ID: {row['id']}\nCategory: {row['category_title']}\nName: {row['name']}\nColor: {row['color']}\nPrice: RM{row['price']}\nDescription: {row['description']}"
    for _, row in df.iterrows()
]

# Step 3: Optional – check total and preview
print(f"Total products: {len(docs)}")
print(docs[2])  # preview first
print(len(docs[2]))

import tiktoken

encoding = tiktoken.encoding_for_model("text-embedding-ada-002")  # or "gpt-3.5-turbo"
token_count = len(encoding.encode(docs[2]))

print("Token count for docs[2]:", token_count)


Total products: 16
Product ID: 3
Category: Tumbler
Name: [Corak Malaysia] Tiga Sekawan Bundle
Color: nan
Price: RM133.9
Description: Show your appreciation of our local arts with the Tiga Sekawan bundle in three variations, inspired by the vibrant culture and history of our country’s thirteen states.

This bundle features:

Corak Malaysia All Day Cup: Our All Day Cups now feature unique designs inspired by our cultural arts, embodying the heart of our country with a special ZUS twist on each variation.

Corak Malaysia Cup Sleeve: This sleeve is a beautiful combination of culture and function. Made from soft and durable synthetic leather with an adjustable strap that can adapt to various cup sizes, along with an interchangeable nylon string to hand-carry or wear crossbody. 

Corak Malaysia Reusable Straw Kit: Sustainable, compact, and designed to be easy to carry. This kit contains a food-grade stainless steel straw with a soft silicone tip to protect your lips and teeth, as well as a b

In [ ]:
import os
import re
import csv
from pinecone import Pinecone, ServerlessSpec
from langchain_community.embeddings import OpenAIEmbeddings 
from langchain.text_splitter import CharacterTextSplitter

# Initialize Pinecone client
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

PRODUCT_CSV_PATH = "data/zus_products.csv"
PINECONE_CLOUD = "aws"
PINECONE_ENVIRONMENT = "us-east-1" # Your Pinecone environment, e.g., 'gcp-starter', 'us-west-2'
INDEX_NAME = "zus-index"
EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_DIMENSION = 1536 # Dimension for text-embedding-3-small

if INDEX_NAME not in pc.list_indexes().names():
    print(f"Index '{INDEX_NAME}' not found. Creating index...")
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBEDDING_DIMENSION,
        metric='cosine',  # 'euclidean' or 'dotproduct' are also options, 'cosine' is common for embeddings
        spec=ServerlessSpec(cloud=PINECONE_CLOUD, region=PINECONE_ENVIRONMENT)
    )
    print(f"Index '{INDEX_NAME}' created successfully.")
else:
    index_stats = pc.describe_index(INDEX_NAME)
    print(f"Index '{INDEX_NAME}' already exists.")
    print("Dimension:", index_stats.dimension)
    print("Metric:", index_stats.metric)
    print("Status:", index_stats.status)


# Initialize embedding model (this part of your code was generally correct
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL, openai_api_key=os.environ.get('OPENAI_API_KEY'))

# You can initialize the PineconeVectorStore later when you have documents to add or query
print("\nScript completed. Ensure your Pinecone API Key and OpenAI API Key are correctly set as environment variables.")

Index 'zus-index' already exists.
Dimension: 1536
Metric: cosine
Status: {'ready': True, 'state': 'Ready'}

Script completed. Ensure your Pinecone API Key and OpenAI API Key are correctly set as environment variables.


In [ ]:
import time
import numpy as np
import uuid # For generating unique IDs for Pinecone
from langchain_core.documents import Document
# from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_products_data(csv_file):
    """
    Loads product data from a CSV, and prepares it for text splitting,
    specifically focusing on the 'description' column while preserving
    other columns as metadata. Converts NaN to empty string for Pinecone compatibility.
    """
    print(f"Loading data from {csv_file}...")
    try:
        df = pd.read_csv(csv_file)
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_file}")
        return []
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return []

    documents_for_splitting = []
    for index, row in df.iterrows():
        metadata = {}
        for col in df.columns:
            if col == 'description':
                continue # description is handled as page_content

            value = row[col]
            # --- CRITICAL CHANGE HERE ---
            # Convert NaN/None to empty string for metadata fields expected to be strings
            if pd.isna(value):
                metadata[col] = "" # Convert to empty string instead of None
            elif isinstance(value, (np.integer, np.floating)):
                # Convert numpy specific integer/float types to standard Python types
                metadata[col] = value.item()
            else:
                # Ensure all other types are string or directly JSON compatible
                metadata[col] = str(value)

        # Ensure price is handled specifically if it came as a string or NaN
        # If price must be a number, handle cases where it's empty string or invalid
        if 'price' in metadata:
            try:
                # Attempt to convert to float. If it was already "", this will likely fail.
                # If it must be a number and cannot be empty, you might skip it or set a default.
                if metadata['price'] == "": # If it's an empty string from prior NaN conversion
                    metadata['price'] = None # Revert to None if numeric field; or 0.0, etc. based on need
                else:
                    metadata['price'] = float(metadata['price'])
            except ValueError:
                metadata['price'] = None # Set to None if it cannot be converted to float


        doc = Document(
            page_content=str(row['description']), # Ensure description is a string
            metadata=metadata
        )
        documents_for_splitting.append(doc)

    print(f"Prepared {len(documents_for_splitting)} documents for splitting.")
    return documents_for_splitting

def ingest_data_to_pinecone_from_csv(csv_file, index_name, embedding_model):
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
    index = pc.Index(index_name) 
    """
    Ingests text documents generated from a CSV into a Pinecone index.
    """

    # Initialize embeddings model
    print(f"Loading embeddings model: {EMBEDDING_MODEL}")

    documents_to_ingest = load_products_data(csv_file)
    if not documents_to_ingest:
        print("No documents to ingest. Exiting ingestion process.")

    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=600,
    #     chunk_overlap=50,
    #     length_function=len,
    # )
    # texts = text_splitter.split_documents(documents_to_split)
    # print(f"Created {len(texts)} chunks after splitting descriptions.")

    # Prepare vectors for upserting
    vectors_to_upsert = []
    for i, doc in enumerate(documents_to_ingest):
        # Generate embedding for the full document content
        embedding = embedding_model.embed_query(doc.page_content)
        # Create a unique ID for each document
        vector_id = str(uuid.uuid4())

        # Use the already processed metadata from the Document object
        # Ensure 'text' (the full document content) is also explicitly in metadata
        final_metadata = doc.metadata.copy() # Make a copy to avoid modifying original doc metadata directly
        final_metadata["text"] = doc.page_content # Store the full document content itself

        vectors_to_upsert.append({
            "id": vector_id,
            "values": embedding,
            "metadata": final_metadata
        })

    # Upsert in batches (Pinecone has limits per request)
    batch_size = 32 # Adjust batch size as needed
    for i in range(0, len(vectors_to_upsert), batch_size):
        batch = vectors_to_upsert[i:i + batch_size]
        try:
            index.upsert(vectors=batch)
            print(f"Upserted batch {i//batch_size + 1}/{(len(vectors_to_upsert) + batch_size - 1) // batch_size}")
        except Exception as e:
            print(f"Error upserting batch starting at index {i}: {e}")
            # Depending on error, you might want to retry or log more details

    print("Data ingestion to Pinecone completed.")
    time.sleep(10)
    print(index.describe_index_stats())


In [65]:
ingest_data_to_pinecone_from_csv(
        PRODUCT_CSV_PATH,
        INDEX_NAME,
        PINECONE_ENVIRONMENT,
        embedding_model,
    )

Loading embeddings model: text-embedding-3-small
Loading data from ../data/zus_products.csv...
Prepared 16 documents for splitting.
Upserted batch 1/1
Data ingestion to Pinecone completed.
{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [ ]:
from fastapi import FastAPI, HTTPException
from langchain.prompts import PromptTemplate
from typing import List, Dict, Any
from typing_extensions import TypedDict
from langchain.chains import LLMChain
from langchain.utilities import SQLDatabase
from langchain.chat_models import ChatOpenAI
import json
import re
from pydantic import BaseModel
import uvicorn

from pinecone import Pinecone
from langchain_community.embeddings import OpenAIEmbeddings 
import os
import sqlite3

import dotenv
dotenv.load_dotenv()

import nest_asyncio
nest_asyncio.apply()

# Configuration

INDEX_NAME = "zus-index"
DEFAULT_TOP_K = 3
MAX_TOP_K_RETRIEVAL = 5 # Set a reasonable upper limit for top_k
EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_DIMENSION = 1536 # Dimension for text-embedding-3-small
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL, openai_api_key=os.environ.get('OPENAI_API_KEY'))
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

OUTLETS_DB_PATH = "data/zus_outlets.db"
OUTLETS_SQL_PATH = "data/zus_outlets.sql"

pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
pinecone_index = None
try:
    if pc and INDEX_NAME in pc.list_indexes().names():
        pinecone_index = pc.Index(INDEX_NAME)
        print(f"Connected to Pinecone index: {INDEX_NAME}")
    elif pc:
        print(f"Pinecone index '{INDEX_NAME}' does not exist. Please run ingestion script.")
except Exception as e:
    print(f"Error connecting to Pinecone index: {e}")

# --- SETUP DATABASE CONNECTION ---
if not os.path.exists(OUTLETS_DB_PATH):
    print(f"Creating SQLite DB at {OUTLETS_DB_PATH} from {OUTLETS_SQL_PATH}...")
    conn = sqlite3.connect(OUTLETS_DB_PATH)
    with open(OUTLETS_SQL_PATH, "r", encoding="utf-8") as f:
        sql_script = f.read()
    conn.executescript(sql_script)
    conn.close()
    print("Database created successfully.")
else:
    print(f"Database already exists at {OUTLETS_DB_PATH}.")

outlets_sql_db = SQLDatabase.from_uri(f"sqlite:///{OUTLETS_DB_PATH}")  # Update with your actual DB path

app = FastAPI(
    title="ZUS Coffee Product & Outlets API (Pinecone & Text2SQL)",
    description="API for retrieving ZUS Coffee product information and outlet details using Pinecone and Text2SQL.",
    version="1.0.0",
)

PRODUCT_SUMMARY_PROMPT_TEMPLATE = """
Here is the user query and relevant text chunks. Your goal is to provide a concise and coherent answer based *only* on the provided text chunks.

---
User Query: {question}

Relevant Text Chunks:
{context}
---

Step 1: Summarize user question in simpler words.
Step 2: Decide which retrieved text chunks directly apply to the simplified question.
Step 3: Combine those directly applicable chunks into a concise outline or key points.
Step 4: Draft a single, coherent answer based *only* on the outline from Step 3.
Step 5: Review the drafted answer for accuracy and coherence, ensuring it directly addresses the user query using only provided information.

Show all steps, then provide a final refined answer.

Final Refined Answer: (Please ensure all prices are stated in RM, the currency unit.)
"""

INTENT_PROMPT_TEMPLATE = """
You are a smart assistant. Classify the user's intent as one of:
- "product"
- "outlet"

Then, identify if any information is missing to complete the task.
User: {input}
Respond in JSON with "intent" and "missing_info".
"""

OUTLET_WRITE_QUERY_PROMPT = """
---
User Query: {question}
---

Given an input question, create a syntactically correct {dialect} SQL query 
to help find the answer. Unless the user specifies a specific number of results, 
always limit your query to at most {top_k} results using LIMIT.

IMPORTANT:
Do not add markdown blocks for the SQL Query.
- Return your answer ONLY as SQL Query, "SELECT ..."
- DO NOT include commentary or explanation.
- DO NOT include raw SQL outside of the JSON format.
- Use only these tables and columns:

{table_info}

Be careful not to use columns that don't exist.
"""

OUTLET_SUMMARY_PROMPT_TEMPLATE = """
Given the following user question, corresponding SQL query, and SQL result, answer the user question concisely.

Question: {question}
SQL Query: {query}
SQL Result: {result}

"""


# --- Integrate the prompt into summary_chain ---
product_summary_prompt = PromptTemplate(
    template=PRODUCT_SUMMARY_PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)
product_summary_chain = LLMChain(llm=llm, prompt=product_summary_prompt) if llm else None

outlet_write_query_prompt = PromptTemplate(
    template=OUTLET_WRITE_QUERY_PROMPT,
    input_variables=["question", "dialect", "top_k", "table_info"]
)
outlet_write_query_chain = LLMChain(llm=llm, prompt=outlet_write_query_prompt) if llm else None
outlet_summary_prompt = PromptTemplate(
    template=OUTLET_SUMMARY_PROMPT_TEMPLATE,
    input_variables=["question", "query", "result"]
)
outlet_summary_chain = LLMChain(llm=llm, prompt=outlet_summary_prompt) if llm else None

intent_prompt = PromptTemplate.from_template(INTENT_PROMPT_TEMPLATE)
intent_chain = LLMChain(llm=llm, prompt=intent_prompt)

class ChatInput(BaseModel): # Input model for the /chat endpoint
    prompt: str

class OutletState(TypedDict):
    question: str = ""
    query: str = ""
    answer: str = ""
    result: List[List[Any]]

class OutletResponse(BaseModel):
    summary: str = ""
    sql_query: str = ""
    executed_sql_result: List[List[Any]]

class RetrievedProductDetail(BaseModel): # New model to structure retrieved product info
    name: str
    category: str
    price: Any # Use Any as price might be float or None
    color: str
    image: str
    snippet: str # The specific text chunk retrieved
    score: float # Similarity score

class ProductResponse(BaseModel):
    summary: str
    retrieved_products: List[RetrievedProductDetail] # List of the new detail model

def detect_intent(prompt: str) -> str:
    """
    Detects intent using keywords first, then falls back to LLM if no match.
    """
    outlet_keywords = ["outlet", "location", "address", "open", "hours", "where", "branch", "store"]
    product_keywords = ["product", "cup", "tumbler", "price", "color", "drink", "menu", "cost", 'item']

    prompt_lower = prompt.lower()
    if any(word in prompt_lower for word in outlet_keywords):
        return "outlet"
    if any(word in prompt_lower for word in product_keywords):
        return "product"
    # --- LLM fallback for intent detection ---
    intent_json = intent_chain.invoke({"input": prompt})
    if not intent_json.strip().startswith("{"):
        print("LLM returned invalid JSON:", intent_json)
        raise HTTPException(status_code=500, detail="Intent detection failed. LLM did not return valid JSON.")
    intent_data = json.loads(intent_json)
    return intent_data

# --- Function to extract top_k from query ---
def extract_top_k_from_query(query: str) -> int:
    """
    Attempts to extract a number for 'top_k' from the user's query.
    Defaults to DEFAULT_TOP_K if no number is found or if it's invalid.
    Caps at MAX_TOP_K_RETRIEVAL.
    """
    match = re.search(r'(?:top|best|any)\s+(\d+)', query, re.IGNORECASE)
    if match:
        try:
            num = int(match.group(1))
            return min(num, MAX_TOP_K_RETRIEVAL) # Cap at max
        except ValueError:
            pass # Fall through to default if conversion fails
    return DEFAULT_TOP_K

def extract_final_answer(full_llm_response) -> str:
    final_answer_tag = "Final Refined Answer:"
    if final_answer_tag in full_llm_response:
        summary = full_llm_response.split(final_answer_tag, 1)[1].strip()
    else:
        # Fallback if the tag is not found (e.g., LLM deviates)
        summary = full_llm_response.strip()
    return summary

def execute_sql_query(state: OutletState):
    """Execute SQL query directly using sqlite3."""
    print("\nExecuting SQL Query:")
    print(state["query"])

    try:
        conn = sqlite3.connect(OUTLETS_DB_PATH)
        cursor = conn.cursor()
        cursor.execute(state["query"])
        rows = cursor.fetchall()
        cursor.close()
        conn.close()

        state["result"] = rows
        return state
    except sqlite3.Error as e:
        print(f" Error: {e}")
        raise e

@app.get("/products", response_model=ProductResponse)
async def get_products(query: str):
    """
    Retrieves and summarizes information about ZUS Coffee drinkware products
    from Pinecone based on a user query.
    """
    if not query:
        raise HTTPException(status_code=400, detail="Query parameter cannot be empty.")
    if embedding_model is None:
        raise HTTPException(status_code=503, detail="Embeddings model not loaded.")
    if pinecone_index is None:
        raise HTTPException(status_code=503, detail="Pinecone index not available. Please ensure ingestion script was run and API keys are correct.")
    if product_summary_chain is None:
         raise HTTPException(status_code=503, detail="Text generation model not loaded. Cannot summarize products.")

    try:
        # Determine top_k dynamically from the query
        actual_top_k = extract_top_k_from_query(query)
        print(f"User query requested top_k: {actual_top_k}")

        # Embed the user query
        query_embedding = embedding_model.embed_query(query)

        # Perform similarity search on Pinecone
        # top_k is the number of similar vectors to retrieve
        results = pinecone_index.query(
            vector=query_embedding,
            top_k=actual_top_k, # Retrieve top 3 similar chunks
            include_metadata=True # Ensure metadata is returned
        )

        if not results.matches:
            return {"query": query, "summary": "No relevant products found."}

        # Combine retrieved content from metadata
        context_docs = []
        retrieved_products_info = [] # List to store product details for API response

        if not results.matches:
            return ProductResponse(summary="No relevant products found.", retrieved_products=[])

        # Combine retrieved content and metadata for LLM context and API response
        for match in results.matches:
            if "text" in match.metadata:
                product_name = match.metadata.get('name', 'Unknown Product')
                category_title = match.metadata.get('category_title', 'N/A')
                product_image = match.metadata.get('image', 'N/A')
                product_price = match.metadata.get('price', 'N/A')
                product_color = match.metadata.get('color', 'N/A') # Assuming color is now "" if NaN

                # Add more metadata to the context for the LLM
                context_docs.append(
                    f"Product Name: {product_name}\n"
                    f"Category: {category_title}\n"
                    f"Colors Available: {product_color}\n"
                    f"Price: {product_price}\n"
                    f"Description Snippet: {match.metadata['text']}"
                )

                # Prepare the detailed product info for the API response
                retrieved_products_info.append(
                    RetrievedProductDetail(
                        name=product_name,
                        category=category_title,
                        price=product_price,
                        color=product_color,
                        image=product_image,
                        snippet=match.metadata['text'],
                        score=match.score
                    )
                )

        if not context_docs:
            summary = "I couldn't find any relevant products based on your query. Please try a different query."
        else:
            context = "\n\n---\n\n".join(context_docs)
            full_llm_response = product_summary_chain.invoke({"context": context, "question": query})['text']
            # --- MODIFICATION START ---
            # Extract only the "Final Refined Answer" part
            summary = extract_final_answer(full_llm_response)
            # --- MODIFICATION END ---
        return ProductResponse(summary=summary, retrieved_products=retrieved_products_info)    
    except Exception as e:
        print(f"Error during product retrieval from Pinecone: {e}")
        raise HTTPException(status_code=500, detail=f"An error occurred during product retrieval: {e}")

@app.get("/outlets", response_model=OutletResponse)
async def get_outlets(query: str):
    if not query:
        raise HTTPException(status_code=400, detail="Query parameter cannot be empty.")
    if not outlet_write_query_chain:
        raise HTTPException(status_code=503, detail="Text generation model not loaded. Cannot generate sql query.")
    if not outlet_summary_chain:
        raise HTTPException(status_code=503, detail="Text generation model not loaded. Cannot summarize products.")
    try:
        actual_top_k = extract_top_k_from_query(query)
        print(f"User query requested top_k: {actual_top_k}")
        state=OutletState({"question": query})
        try:
            response = outlet_write_query_chain.invoke({"question": state["question"], 
                                                "top_k": actual_top_k, 
                                                "dialect": outlets_sql_db.dialect, 
                                                "table_info": outlets_sql_db.get_table_info()})
            print("Response of generating sql query: ",response)
            state["query"] = response['text']
        except Exception as e:
            print(f"Error writing query: {e}")
            raise HTTPException(status_code=500, detail="An error occurred while writing query.")
        try:
            state=execute_sql_query(state)
        except Exception as e:
            print(f"Error executing query: {e}")
            raise HTTPException(status_code=500, detail="An error occurred while executing query.")
        if len(state['result'])==0:
            state["answer"] = "I couldn't find any relevant outlets based on your query. Please try a different query."
        else:
            try:
                response=outlet_summary_chain.invoke({"question": state["question"], "query": state["query"], "result": state["result"]})
                state["answer"]=response['text']
            except Exception as e:
                print(f"Error generating answer: {e}")
                raise HTTPException(status_code=500, detail="An error occurred while generating answer.")
        return OutletResponse(summary=state["answer"], sql_query=state["query"], executed_sql_result=state["result"])
    except Exception as e:
        print(f"Error translating/executing query: {e}")
        raise HTTPException(status_code=500, detail="An error occurred while querying outlet data.")


@app.post("/chat")
async def chat_endpoint(chat_input: ChatInput):
    prompt = chat_input.prompt
    if not prompt:
        raise HTTPException(status_code=400, detail="Prompt cannot be empty.")

    # Use LLM to classify intent
    try:
        intent = detect_intent(prompt)
        print(f"Intent: {intent}")
        if type(intent) == dict:
            intent = intent.get("intent")
            missing_info = intent.get("missing_info", "")
            if missing_info:
                print(f"Summary: I need more information to proceed: {missing_info}")
                return f"I need more information: {missing_info}"
            try:
                if intent == "product":
                    response = await get_products(prompt)
                    return response
                elif intent == "outlet":
                    response = await get_outlets(prompt)
                    return response
                else:
                    raise HTTPException(status_code=400, detail="Could not determine query type.")
            except Exception as e:
                print(f"Error in chat endpoint: {e}")
                raise HTTPException(status_code=500, detail="An error occurred in the chat service.")
        elif intent == "product":
            response = await get_products(prompt)
            return response
        elif intent == "outlet":
            response = await get_outlets(prompt)
            return response
    except Exception as e:
        print(f"Intent classification error: {e}")
        raise HTTPException(status_code=500, detail="Could not classify intent.")


if __name__ == "__main__":
    print("Starting FastAPI app in Jupyter Notebook...")
    uvicorn.run(app, host="127.0.0.1", port=8000)


Task exception was never retrieved
future: <Task finished name='Task-52' coro=<Server.serve() done, defined at c:\Users\User\anaconda3\envs\rag-chatbot-agent\Lib\site-packages\uvicorn\server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "c:\Users\User\anaconda3\envs\rag-chatbot-agent\Lib\site-packages\uvicorn\main.py", line 580, in run
    server.run()
  File "c:\Users\User\anaconda3\envs\rag-chatbot-agent\Lib\site-packages\uvicorn\server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "C:\Users\User\AppData\Roaming\Python\Python312\site-packages\nest_

Connected to Pinecone index: zus-index
Database already exists at ../data/zus_outlets.db.
Starting FastAPI app in Jupyter Notebook...


INFO:     Started server process [29728]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Intent: outlet
User query requested top_k: 3
Response of generating sql query:  {'question': 'Which outlets in Selangor open after 9pm?', 'top_k': 3, 'dialect': 'sqlite', 'table_info': '\nCREATE TABLE outlets (\n\tid INTEGER, \n\tname TEXT, \n\taddress TEXT, \n\tlink TEXT, \n\treviews_count INTEGER, \n\treviews_average FLOAT, \n\tphone_number TEXT, \n\tservices TEXT, \n\tplace_type TEXT, \n\topens_at TEXT, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from outlets table:\nid\tname\taddress\tlink\treviews_count\treviews_average\tphone_number\tservices\tplace_type\topens_at\n1\tZUS Coffee – Temu Business Centre City Of Elmina\tNo 5 (Ground Floor), Jalan Eserina AA U16/AA Elmina, East, Seksyen U16, 40150 Shah Alam, Selangor\thttps://maps.app.goo.gl/XduXnuUs4E2a1VE56\t82\t4.1\t012-816 1340\tDine-in, Kerbside pickup, Delivery\tCafe\tMonday, 8am–9:40pm, Tuesday, 8am–9:40pm, Wednesday, 8am–9:40pm, Thursday, 8am–9:40pm, Friday, 8am–9:\n2\tZUS Coffee – Spectrum Shopping Mall\tLot CW-5 Cafe Walk, Ground

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [29728]


## Prompt Tests for `/products` (Happy Flows)
<table> <thead> <tr> <th>Test Case</th> <th>Prompt Example</th> </tr> </thead> <tbody> <tr> <td>Normal query</td> <td>Show me the top 3 black drinkware items.</td> </tr> <tr> <td>Price-based query</td> <td>Which ZUS tumblers are under RM50?</td> </tr> <tr> <td>Style-based query</td> <td>List elegant glass cups for gifts.</td> </tr> <tr> <td>Ranking-based query</td> <td>Give me the best 2-selling items in the Accessories category.</td> </tr> </tbody> </table>

## Prompt Tests for `/outlets` (Happy Flows)
<table> <thead> <tr> <th>Test Case</th> <th>Prompt Example</th> </tr> </thead> <tbody> <tr> <td>Location-specific</td> <td>Where are ZUS Coffee outlets in Petaling Jaya?</td> </tr> <tr> <td>Time-specific</td> <td>Which outlets in Selangor open after 9pm?</td> </tr> <tr> <td>Feature-specific</td> <td>List outlets with self-pickup in Kuala Lumpur.</td> </tr> <tr> <td>Parking-related</td> <td>Show outlets with parking near Damansara.</td> </tr> </tbody> </table>

## Prompt Tests for Unhappy Flows
1. Missing Parameters
<table> <thead> <tr> <th>Endpoint</th> <th>Prompt Example</th> <th>Expected Behavior</th> </tr> </thead> <tbody> <tr> <td>/products</td> <td>Show products</td> <td>HTTP 400 with "Query parameter cannot be empty."</td> </tr> <tr> <td>/products</td> <td>Calculate</td> <td>HTTP 400 with "Query parameter cannot be empty."</td> </tr> <tr> <td>/outlets</td> <td>Show outlets</td> <td>HTTP 400 with "Query parameter cannot be empty."</td> </tr> <tr> <td>/outlets</td> <td>Where</td> <td>HTTP 400 with "Query parameter cannot be empty."</td> </tr> <tr> <td>/chat</td> <td>{ "prompt": "" }</td> <td>HTTP 400 with "Prompt cannot be empty."</td> </tr> </tbody> </table>

2. API Downtime (Simulated)
<table> <thead> <tr> <th>Endpoint</th> <th>How to Simulate</th> <th>Expected Behavior</th> </tr> </thead> <tbody> <tr> <td>/products</td> <td>Set <code>embedding_model = None</code></td> <td>HTTP 503 with "Embeddings model not loaded."</td> </tr> <tr> <td>/products</td> <td>Raise <code>Exception("Test 500")</code></td> <td>HTTP 500 with "An error occurred during product retrieval."</td> </tr> <tr> <td>/outlets</td> <td>Set <code>outlets_chain = None</code></td> <td>HTTP 503 with "Text generation model not loaded."</td> </tr> </tbody> </table>

3. Malicious Payload (SQL Injection)
<table> <thead> <tr> <th>Endpoint</th> <th>Prompt Example</th> <th>Expected Behavior</th> </tr> </thead> <tbody> <tr> <td>/outlets</td> <td>Which outlets are open at ' OR '1'='1</td> <td>HTTP 500 with "An error occurred while querying outlet data."</td> </tr> <tr> <td>/outlets</td> <td>DROP TABLE outlets; --</td> <td>HTTP 500 with no system crash, and no destructive execution</td> </tr> <tr> <td>/outlets</td> <td>List outlets where location = "" OR 1=1</td> <td>Handled securely, no full data dump</td> </tr> </tbody> </table>

# API Test Results

## Product API Test Results

![Product API Test 1](../assets/ProductAPITest1.jpg)

![Product API Test 2](../assets/ProductAPITest2.jpg)

![Product API Test 3](../assets/ProductAPITest3.jpg)

![Product API Test 4](../assets/ProductAPITest4.jpg)

## Outlet API Test Results

![Outlet API Test 1](../assets/OutletAPITest1.jpg)

![Outlet API Test 2](../assets/OutletAPITest2.jpg)

![Outlet API Test 3](../assets/OutletAPITest3.jpg)

![Outlet API Test 4](../assets/OutletAPITest4.jpg)